# Using LSTM for Predicting change in Sentiments

Here, I use an LSTM model to predict the change in the sentiment of an incoming utterance given the previous sentiment. This model takes as input word sequences and a real number for the sentiment of previous utterance. This model is able to take word ordering into account. Similar to the previous models, I use a pre-trained word embeddings to represent words, 50 dimensional Glove. For my implementation I use Keras with tensorflow as the backend.

## Overview of the model


In [46]:
from utility import *

from keras.models import Model
from keras.layers import Dense, Input, Dropout, LSTM, Activation, Concatenate
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

np.random.seed(1)

In [4]:
# 50-dimensional GloVe embeddings
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B/glove.6B.50d.txt')

In [25]:
def sentences_to_indices(samples, word_to_index, max_len):
    
    m = len(samples)                                   # number of training examples
    X_indices = np.zeros((m, max_len))
    
    for i, sentence in enumerate(samples):                               # loop over training examples
        
        sentence = re.sub(r'[^\w\s]', ' ', sentence.strip())  
        words = [i.lower() for i in sentence.strip().split()]
        j = 0
        
        for w in words:
            if w in word_to_index:
                X_indices[i, j] = word_to_index[w]
            else:
                X_indices[i, j] = word_to_index['unk']
            j += 1
            if j == max_len: break
                
    return X_indices

In [64]:
def create_features(samples_list):
    
    X, Y, aux_X = [], [], []
    for sample in samples_list:            
        X.append(sample['utterance'])
        Y.append(sample['current_emotion'] - sample['prev_emotion'])
        aux_X.append(sample['prev_emotion'])
    
    return X, aux_X, Y

In [14]:
def pretrained_embedding_layer(word_to_vec_map, word_to_index):
   
    vocab_len = len(word_to_index) + 1                  # adding 1 to fit Keras embedding (requirement)
    emb_dim = word_to_vec_map["cucumber"].shape[0]      # define dimensionality of your GloVe word vectors (= 50)
    emb_matrix = np.zeros((vocab_len, emb_dim))
    
    for word, index in word_to_index.items():
        emb_matrix[index, :] = word_to_vec_map[word]

    embedding_layer = Embedding(vocab_len, emb_dim, trainable=False)
    embedding_layer.build((None,))
    embedding_layer.set_weights([emb_matrix])
    
    return embedding_layer

In [54]:
def sentiment_model(input_shape, word_to_vec_map, word_to_index):
    
    sentence_indices = Input(input_shape, dtype='int32')
    aux_input = Input((1, ), dtype='float32')
    
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)
    embeddings = embedding_layer(sentence_indices)   
    
    X = LSTM(128, return_sequences=True)(embeddings)
    X = Dropout(0.5)(X)
    X = LSTM(128, return_sequences=False)(X)
    X = Dropout(0.5)(X)
    X = Dense(1)(X)
    
    merged = Concatenate()([aux_input, X])
    merged = Dense(1)(merged)
    output = Activation('tanh')(merged)
    model = Model(inputs=[aux_input, sentence_indices], outputs=output)
        
    return model

In [16]:
train_conversations = load_conversations(category='train')
find_sentiments(train_conversations)

test_conversations = load_conversations(category='test')
find_sentiments(test_conversations)

validation_conversations = load_conversations(category='validation')
find_sentiments(validation_conversations)

In [19]:
maxLen = 15

In [58]:
train_samples = create_samples(train_conversations)
x_train, aux_x_train, y_train = create_features(train_samples)

In [71]:
print(len(x_train), len(aux_x_train), len(y_train))

76052 76052 76052


In [65]:
aux_x_train = np.array(aux_x_train) 
aux_x_train.reshape(-1, 1)
print(aux_x_train.shape)

(76052, 1)


In [55]:
model = sentiment_model((maxLen,), word_to_vec_map, word_to_index)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_9 (InputLayer)            (None, 15)           0                                            
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 15, 50)       20000050    input_9[0][0]                    
__________________________________________________________________________________________________
lstm_11 (LSTM)                  (None, 15, 128)      91648       embedding_6[0][0]                
__________________________________________________________________________________________________
dropout_11 (Dropout)            (None, 15, 128)      0           lstm_11[0][0]                    
__________________________________________________________________________________________________
lstm_12 (L

In [67]:
model.compile(loss='mse', optimizer='adam', metrics=['mse'])
X_train_indices = sentences_to_indices(x_train, word_to_index, maxLen)
model.fit([aux_x_train, x_train_indices], y_train, epochs=10, batch_size=32, shuffle=True)

Epoch 1/10
76052/76052 [==============================] - 138s 2ms/step - loss: 0.1026 - mean_squared_error: 0.1026
Epoch 2/10
76052/76052 [==============================] - 139s 2ms/step - loss: 0.0373 - mean_squared_error: 0.0373
Epoch 3/10
76052/76052 [==============================] - 147s 2ms/step - loss: 0.0299 - mean_squared_error: 0.0299
Epoch 4/10
76052/76052 [==============================] - 145s 2ms/step - loss: 0.0259 - mean_squared_error: 0.0259
Epoch 5/10
76052/76052 [==============================] - 136s 2ms/step - loss: 0.0231 - mean_squared_error: 0.0231
Epoch 6/10
76052/76052 [==============================] - 150s 2ms/step - loss: 0.0209 - mean_squared_error: 0.0209
Epoch 7/10
76052/76052 [==============================] - 144s 2ms/step - loss: 0.0189 - mean_squared_error: 0.0189
Epoch 8/10
76052/76052 [==============================] - 133s 2ms/step - loss: 0.0174 - mean_squared_error: 0.0174
Epoch 9/10
76052/76052 [==============================] - 134s 2ms/step 

In [69]:
test_samples = create_samples(test_conversations)
x_test, aux_x_test, y_test = create_features(test_samples)
aux_x_test = np.array(aux_x_test) 
aux_x_test.reshape(-1, 1)

array([[0.  ],
       [0.  ],
       [0.  ],
       ...,
       [0.2 ],
       [0.  ],
       [0.65]])

In [70]:
x_test_indices = sentences_to_indices(x_test, word_to_index, max_len=maxLen)
loss, metric = model.evaluate([aux_x_test, x_test_indices], y_test)
print("Mean square error = ", loss)

6740/6740 [==============================] - 4s 566us/step
Mean square error =  0.019441413279700174
